Evaluate the LLM responses

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [82]:
import json
import os
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings, Document, VectorStoreIndex, get_response_synthesizer, PromptTemplate
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from dotenv import load_dotenv
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re

### Load environment variables (for API keys)

In [31]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)
client = OpenAI(model="gpt-4o-mini", temperature=0)

### Initialize OpenAI client and Pinecone

In [5]:
embedding = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.llm = client
Settings.embed_model = embedding
Settings.chunk_size_limit = 1536

In [11]:
# Initialize Pinecone vector store (if required)
pinecone_index = pinecone_client.Index("chatbot-index")
vector_store = PineconeVectorStore(pinecone_index)
retriever = VectorIndexRetriever(index=VectorStoreIndex.from_vector_store(vector_store=vector_store), similarity_top_k=5)

In [12]:
# Define the chatbot's prompt template
prompt_template = (
    "You are a friendly chatbot specialized in helping beginners use FamilySearch and its tools. 😊 "
    "This includes record hints, source attachments, and other related topics. Provide clear and concise answers, "
    "and try to make the conversation enjoyable! 😄\n\n"
    "Context:\n"
    "#####################################\n"
    "{context_str}\n"
    "Answer the user's question: {query_str}\n\n"
    "If the question is related to FamilySearch or its tools (such as record hints, source attachments, or genealogical research), "
    "provide a detailed answer along with a summary. Also, include the following source metadata as 'Source':\n"
    "- **Title**: {title}\n"
    "- **Publish Date**: {date}\n"
    "- **URL**: {url}\n\n"
    "However, if the question is unrelated to FamilySearch, provide a direct and concise answer without any summary or metadata."
)

# Create the template and response synthesizer
qa_template = PromptTemplate(template=prompt_template)
response_synthesizer = get_response_synthesizer(
    llm=client, text_qa_template=qa_template, response_mode="compact"
)
query_engine = RetrieverQueryEngine(
    retriever=retriever, response_synthesizer=response_synthesizer
)

In [32]:
# Function to query the LLM for an answer based on a given question
def query_llm(question):
    # Query the LLM using llama_index's query engine
    try:
        response = query_engine.query(question)
        return response.response  # Ensure this extracts the correct text part of the response
    except Exception as e:
        return f"Error fetching LLM response: {str(e)}"

### Function to calculate the similarity score between two strings and to evaluate LLM responses against predefined answers

In [69]:
# Function to calculate semantic similarity using embeddings
def calculate_semantic_similarity(stored_answer, llm_answer):
    # Wrap the answers in Document objects
    stored_doc = Document(content=stored_answer)
    llm_doc = Document(content=llm_answer)

    # Generate embeddings for both answers
    stored_embedding = Settings.embed_model.embed([stored_doc])[0]  # Assuming this is the correct method
    llm_embedding = Settings.embed_model.embed([llm_doc])[0]  # Assuming this is the correct method

    # Calculate cosine similarity
    return cosine_similarity([stored_embedding], [llm_embedding])[0][0] * 100  # Multiply by 100 to get a percentage

In [14]:
# Load JSON file with predefined questions and answers
with open('../data/faq_test_data/faq_test_data2.json', 'r') as f:
    test_data = json.load(f)

In [77]:
# Create a DataFrame from JSON
df = pd.DataFrame(test_data)
df

,question,answer,url
0,What are record hints in Family Tree?,Record hints are historical records that Famil...,https://www.familysearch.org/en/help/helpcente...
1,What is the difference between the left and th...,The left column shows you details from the his...,https://www.familysearch.org/en/help/helpcente...
2,What do the Details and Compare buttons do in ...,The Details button expands both the record det...,https://www.familysearch.org/en/help/helpcente...
3,What is my source box?,Your source box contains a list of sources tha...,https://www.familysearch.org/en/help/helpcente...
4,How do I give feedback to FamilySearch about S...,We want to know what you think about Source Li...,https://www.familysearch.org/en/help/helpcente...
5,I'm stumped. How can I get help with Source Li...,The Source Linker Learning Center is a great f...,https://www.familysearch.org/en/help/helpcente...
6,Why is attaching a source important?,Sources help answer questions and make informa...,https://www.familysearch.org/en/help/helpcente...
7,What do I do with an unfinished attachment?,An unfinished attachment is a source that has ...,https://www.familysearch.org/en/help/helpcente...


In [88]:
# List to store LLM responses and expected answers
llm_responses = []
expected_answers = []

In [89]:
# Iterate through the DataFrame to compare the LLM responses with the stored answers
for index, row in df.iterrows():
    question = row['question']
    expected_answer = row['answer']

    # Get the LLM's answer
    llm_response = query_llm(question)

    # Extract the answer from the LLM response
    llm_answer = llm_response.split("Source:")[0].strip()  # Get everything before "Source:"

    # Append results
    llm_responses.append(llm_answer)
    expected_answers.append(expected_answer)

In [90]:

# Add the results to the DataFrame
df['LLM Response'] = llm_responses
df['Expected Answer'] = expected_answers

In [93]:
# Configure pandas display settings
pd.set_option('display.max_colwidth', None)  # Show full column width
pd.set_option('display.max_rows', None)   

In [94]:
# Display the relevant columns of the DataFrame
df[['question', 'Expected Answer', 'LLM Response']]

,question,Expected Answer,LLM Response
0,What are record hints in Family Tree?,"Record hints are historical records that FamilySearch finds for you about your ancestors. These hints can often help you identify missing children, parents, siblings, and other relatives in your family tree. Learn more about record hints.","Record hints in Family Tree are historical records that FamilySearch automatically finds for you about your ancestors. This hinting system is continuously searching for new records, as FamilySearch regularly adds historical data. \n\nThese hints can provide valuable information about your ancestors, such as their occupations, physical descriptions, and even details about missing family members like children, parents, and siblings. You can easily transfer information from these record hints to Family Tree without needing to retype it.\n\nTo make the most of record hints, you should:\n\n1. Verify that the record hint is indeed about your ancestor.\n2. Transfer any desired information from the record hint to Family Tree. Note that you cannot replace existing information directly on the website; you'll need to update it manually in Family Tree. However, you can replace information using the mobile app.\n3. Attach the record hint to your ancestors in Family Tree, which saves the record as a source for their profiles.\n\nIn summary, record hints are a fantastic way to uncover more about your family history and enhance your Family Tree with accurate information!\n\n**Source**:\n- **Title**: What are record hints in Family Tree?\n- **Publish Date**: April 24, 2024\n- **URL**: [FamilySearch Help Center](https://www.familysearch.org/en/help/helpcenter/article/what-are-record-hints-in-family-tree)"
1,What is the difference between the left and the right column in Source Linker?,The left column shows you details from the historical record. The right column shows you details from the ancestor's person page in the FamilySearch Family Tree.,"In Source Linker, the left column displays details from the historical record you are viewing, while the right column shows information from the ancestor’s person page in Family Tree. If the ancestor is not yet in Family Tree, you will have the option to add them. The purpose of Source Linker is to help you attach records to existing ancestors, ensuring that the details in Family Tree are accurate and supported by evidence.\n\n**Summary**: \n- **Left Column**: Historical record details.\n- **Right Column**: Ancestor’s details from Family Tree (or option to add if not present).\n\n**Source**:\n- **Title**: Understanding the Source Linker screen layout\n- **Publish Date**: June 06, 2024\n- **URL**: [Understanding the Source Linker screen layout](https://www.familysearch.org/en/help/helpcenter/article/understanding-the-source-linker-screen-layout)"
2,What do the Details and Compare buttons do in Source Linker?,"The Details button expands both the record details (left column) and the person details (right column) The Compare button also expands the record details and the person details. In addition, if information in the record is not already in Family Tree, you will see green Add buttons that allow you to add the information.","In Source Linker, the **Details** and **Compare** buttons serve important functions:\n\n- The **Details** button allows you to view more information about a specific person in the Family Tree. When you click on it, you can see the details associated with that ancestor, which helps you understand their profile better.\n\n- The **Compare** button, represented by a purple icon, is used to compare the information from the historical record (left column) with the details already in the Family Tree (right column). When you click this button, it expands both columns so you can see the details side by side. This is particularly useful for determining if the person in the record is the same as the one in Family Tree and for identifying any new or different information that can 